# MNIST 숫자 이미지 분류 (CNN)

### CNN (Convolutional Neural Network, 합성곱 신경망) = Convnet

In [1]:
import numpy as np
import pandas as pd

import keras
import tensorflow as tf
from IPython.display import display
import PIL

# Check if the code is running on GPU or CPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Check if Keras is using GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12145217744503498083
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3198956339
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14384788331509658802
physical_device_desc: "device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Instructions for updating:
Colocations handled automatically by placer.


Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) 크기의 3D 텐서이다.

높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있다.

채널의 수는 Conv2D 층에 전달된 첫 번째 매개 변수에 의해 조절된다. (32개 또는 64개)

마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 Fully Connected Network에 주입한다.

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### 컨브넷 위에 분류기 추가하기

10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 Softmax 활성화 함수를 사용한다.

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

### MNIST 이미지에 컨브넷 훈련하기

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 4s 0us/step


In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 12s 204us/step - loss: 0.1958 - acc: 0.9393
Epoch 2/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.0497 - acc: 0.9848
Epoch 3/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.0343 - acc: 0.9894
Epoch 4/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.0259 - acc: 0.9917
Epoch 5/5
60000/60000 [==============================] - 9s 155us/step - loss: 0.0200 - acc: 0.9940


### 테스트 데이터에서 모델 평가해보기

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 90us/step


In [9]:
test_acc

0.9916

### Detail

위의 예시에서 첫번째 합성곱 층이 (28, 28, 1) 크기의 특성 맵을 입력으로 받아 (26, 26, 32) 크기의 특성 맵을 출력한다.

즉, 입력에 대해 32개의 필터를 적용한다.

32개의 출력 채널 각각은 26 X 26 크기의 배열 값을 가진다.

이 값은 입력에 대한 필터의 Response Map이다.

**Parameter**
- 입력으로부터 뽑아낼 패치의 크기
    전형적으로 3X3 또는 5X5 크기를 사용한다. (위 예시에서는 3X3 크기를 사용)
    

- 특성맵의 출력 깊이
    합성곱으로 계산할 필터의 수이다. (위 예시에서는 깊이 32로 시작해서 깊이 64로 끝)

3D 입력 특성 맵 위를 3X3 또는 5X5 크기의 윈도우가 Sliding하면서 모든 위치에서

3D 특성 패치((window_height, window_width, input_depth) 크기)를 추출하는 방식으로 합성곱이 작동한다.

이런 3D 패치는 (output_depth,) 크기의 1D 벡터로 변환된다.

(Convolution Kernel(합성곱 커널)이라고 불리는 하나의 학습된 가중치 행렬과의 텐서 곱셈을 통하여 변한된다.)

변횐된 모든 벡터는 (height, width, output_depth) 크기의 3D 특성 맵으로 재구성된다.
